## Módulo 2 Análisis y Reporte sobre el desempeño del modelo. (Portafolio Análisis)

Diego Elián Rodríguez Cantú A00829925

## Dataset Utilizado

**Nombre del Dataset:** SMS Spam Collection Dataset

**Enlace al Dataset:** [SMS Spam Collection Dataset](https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset)

**Descripción de los Datos:**

- Cantidad de Registros/Muestras: El dataset contiene un total de 5,572 mensajes de texto etiquetados como spam o no spam ("ham").

- Número de Características: Las características se derivan de la representación de texto en forma de vectores de términos (TF-IDF), por lo que el número de características depende de la dimensionalidad del espacio de términos. En este caso, se han utilizado un máximo de 10,000 términos, lo que significa que cada mensaje se representa mediante un vector de características de longitud 10,000.

- Número de Clases de Salida: El problema es de clasificación binaria, por lo que hay dos clases de salida: "spam" y "no spam" (o "ham").


In [1]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('wordnet')
# from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('omw-1.4')
import pickle
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn import metrics
from sklearn.naive_bayes import GaussianNB

mssgdata = pd.read_csv('spam.csv', encoding='latin-1')
X, y = mssgdata.v2, mssgdata.v1

documents = []

from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
for sen in range(0, len(X)):
    document = re.sub(r'\W', ' ', str(X[sen]))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.lower()
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)
    
vectorizer = CountVectorizer(max_features=10000, min_df=1, max_df=0.6, ngram_range=(1,2), stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\elian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\elian\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Separación y evaluación del modelo con un conjunto de prueba y un conjunto de validación (Train/Test/Validation).

In [2]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=0)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)


### Entrenamiento y evaluación del modelo base.

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Entrenar el modelo de Random Forest con el conjunto de entrenamiento
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train)

y_pred_test = classifier.predict(X_test)

# Calcular el accuracy en el conjunto de prueba
accuracy_base = accuracy_score(y_test, y_pred_test)
print(f'Precisión del modelo base en el conjunto de prueba: {accuracy_base:.2f}')


Precisión del modelo base en el conjunto de prueba: 0.98


In [4]:
print(confusion_matrix(y_test,y_pred_test))
print(classification_report(y_test,y_pred_test))
print(accuracy_score(y_test, y_pred_test))

[[467   0]
 [ 12  78]]
              precision    recall  f1-score   support

         ham       0.97      1.00      0.99       467
        spam       1.00      0.87      0.93        90

    accuracy                           0.98       557
   macro avg       0.99      0.93      0.96       557
weighted avg       0.98      0.98      0.98       557

0.9784560143626571


### Diagnóstico del sesgo y la varianza.

- Sesgo: El sesgo se refiere a la diferencia entre las predicciones del modelo y los valores reales en el conjunto de entrenamiento. Si el sesgo es alto, el modelo subajusta los datos.
- Varianza: La varianza se refiere a la sensibilidad del modelo a pequeñas variaciones en los datos de entrenamiento. Si la varianza es alta, el modelo sobreajusta los datos.

   

Para diagnosticar el sesgo y la varianza del modelo, podemos utilizar las métricas del informe de clasificación, especialmente la precisión y el f1-score. 

- Precisión (Exactitud):
  - Para la clase 'ham': 0.97
  - Para la clase 'spam': 1.00

El modelo tiene una alta precisión tanto para la clase 'ham' como para la clase 'spam'. Esto sugiere que el modelo tiende a hacer predicciones precisas para ambas clases en el conjunto de prueba.

- Recall (Recuperación):
  - Para la clase 'ham': 1.00
  - Para la clase 'spam': 0.87

El modelo tiene un recall perfecto (1.00) para la clase 'ham', lo que significa que identifica correctamente todos los casos de 'ham' en el conjunto de prueba. Sin embargo, el recall para la clase 'spam' es de 0.87, lo que indica que el modelo no logra identificar todos los casos de 'spam'.

- F1-Score:
  - Para la clase 'ham': 0.99
  - Para la clase 'spam': 0.93

El F1-score es una medida que combina la precisión y el recall. Para ambas clases, el F1-score es alto, lo que sugiere un buen equilibrio entre precisión y recall.

Conclusión:

- El modelo tiene un sesgo bajo para la clase 'ham' debido a su alta precisión y recall.
- El modelo tiene un sesgo moderado para la clase 'spam' debido a su alta precisión pero recall no perfecto.
- La varianza del modelo parece ser moderada, ya que las métricas no indican overfitting ni un underfitting significativo.

En resumen, el modelo base parece funcionar bastante bien para la clasificación de mensajes 'ham' y 'spam', pero podría mejorarse para la clase 'spam' mejorando el recall. Esto podría lograrse mediante la optimización de hiperparámetros o la incorporación de técnicas de regularización.

### Optimización del modelo mediante técnicas de regularización.

Una técnica común para reducir la varianza y evitar el sobreajuste es ajustar los parámetros del modelo. En el caso de Random Forest, puedes ajustar los hiperparámetros como 'n_estimators', 'max_depth', 'min_samples_split', etc., utilizando la validación cruzada.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definir la cuadrícula de hiperparámetros a explorar
param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [None, 10, 20, 30],
}


grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=0, min_samples_split=2),
                           param_grid=param_grid,
                           cv=5,  # 5-fold cross-validation
                           scoring='accuracy',
                           verbose= 1)

grid_search.fit(X_train, y_train)

best_classifier = grid_search.best_estimator_

y_pred_val = best_classifier.predict(X_val)
accuracy_optimized = accuracy_score(y_val, y_pred_val)
print(f'Precisión del modelo optimizado en el conjunto de validación: {accuracy_optimized:.2f}')


Fitting 5 folds for each of 12 candidates, totalling 60 fits


: 

: 